In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf

In [2]:
from tensorflow.keras import Sequential

In [3]:
from keras_preprocessing.image import ImageDataGenerator

In [4]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [6]:
from tensorflow.keras import regularizers, optimizers

In [7]:
def append_ext(fn):
    return fn+".png"

def append_extr(fnr):
    return fnr+".jpg"

In [8]:
traindf=pd.read_csv(r"C:\Users\M1048974\Desktop\object_detection\train\trainLabels.csv",dtype=str,encoding = "latin-1")
testdf=pd.read_csv(r"C:\Users\M1048974\Desktop\object_detection\object\test_data.csv",dtype=str,encoding = "latin-1")

In [9]:
traindf.shape

(50000, 2)

In [10]:
traindf["id"]=traindf["id"].apply(append_ext)
testdf["index"] = testdf["index"].apply(append_extr)

In [11]:
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.10)

In [12]:
datagen


In [13]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=r"C:\Users\M1048974\Desktop\object_detection\train\train_data\train_data_1",
x_col="id",
y_col="label",
subset="training", # that means which subset we r using that will be validation or either training here we r using training
batch_size=90, #size of the batches of data (default: 32).
seed=42, # optional random seed for shuffling and transformations.
shuffle=True, # whether to shuffle the data (default: True)
class_mode="categorical",   # One of "categorical", "binary", "sparse", "input", or None. Default: "categorical". 
                            # Determines the type of label arrays that are returned: - "categorical" 
                            # will be 2D one-hot encoded labels, 
target_size=(32,32)) #Tuple of integers (height, width), defaults to (256,256). 
                        # The dimensions to which all images found will be resized.

Found 45000 validated image filenames belonging to 10 classes.


In [14]:
train_generator

In [15]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=r"C:\Users\M1048974\Desktop\object_detection\train\train_data\train_data_1",
x_col="id",
y_col="label",
subset="validation",
batch_size=50,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 5000 validated image filenames belonging to 10 classes.


In [16]:
valid_generator

In [17]:
test_datagen=ImageDataGenerator(rescale=1./255.)

In [18]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory= r"C:\Users\M1048974\Desktop\object_detection\object\test_data_folder",
x_col="index",
y_col=None,
batch_size= 15,
seed=42,
shuffle=False,
class_mode=None,
validate_filenames=False,
target_size=(32,32))

Found 1365 non-validated image filenames.


In [19]:
test_generator

In [20]:
model = Sequential()
model.add(Conv2D(32, (3, 3),activation="relu" ,padding='same',input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, (3, 3), activation = "relu" ,padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(128, (3, 3), activation = "relu" ,padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))
model.compile(optimizers="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [21]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [22]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
500/500 [==============================] - 97s 194ms/step - loss: 1.6980 - accuracy: 0.3849 - val_loss: 1.3327 - val_accuracy: 0.5334
Epoch 2/10
500/500 [==============================] - 91s 181ms/step - loss: 1.2916 - accuracy: 0.5363 - val_loss: 1.2009 - val_accuracy: 0.5826
Epoch 3/10
500/500 [==============================] - 93s 185ms/step - loss: 1.1117 - accuracy: 0.6060 - val_loss: 1.0204 - val_accuracy: 0.6442
Epoch 4/10
500/500 [==============================] - 92s 184ms/step - loss: 0.9946 - accuracy: 0.6516 - val_loss: 0.8090 - val_accuracy: 0.7276
Epoch 5/10
500/500 [==============================] - 93s 185ms/step - loss: 0.9236 - accuracy: 0.6790 - val_loss: 0.8339 - val_accuracy: 0.7076
Epoch 6/10
500/500 [==============================] - 95s 190ms/step - loss: 0.8705 - accuracy: 0.6980 - val_loss: 0.7848 - val_accuracy: 0.7324
Epoch 7/10
500/500 [==============================] - 90s 181ms/step - loss: 0.8292 - accuracy: 0.7122 - val_loss: 0.7311 - val_ac

In [23]:
model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_VALID)

[0.7333590403199196, 0.7652]

In [24]:
test_generator.reset()
predictions = model.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose =1)

91/91 [==============================] - 3s 31ms/step


In [25]:
predicted_class_indices=np.argmax(predictions,axis=1)

In [26]:
train_data_aug = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    horizontal_flip=True,
                    validation_split=0.10)

In [27]:
train_generator=train_data_aug.flow_from_dataframe(
dataframe=traindf,
directory=r"C:\Users\M1048974\Desktop\object_detection\train\train_data\train_data_1",
x_col="id",
y_col="label",
subset="training", # that means which subset we r using that will be validation or either training here we r using training
batch_size=90, #size of the batches of data (default: 32).
shuffle=True, # whether to shuffle the data (default: True)
class_mode="categorical",   # One of "categorical", "binary", "sparse", "input", or None. Default: "categorical". 
target_size=(32,32))       # Determines the type of label arrays that are returned: - "categorical" 
                            # will be 2D one-hot encoded labels, 

Found 45000 validated image filenames belonging to 10 classes.


In [28]:
train_data_aug = ImageDataGenerator(rescale=1./255.,validation_split=0.10)

In [29]:
valid_generator_aug=train_data_aug.flow_from_dataframe(
dataframe=traindf,
directory=r"C:\Users\M1048974\Desktop\object_detection\train\train_data\train_data_1",
x_col="id",
y_col="label",
subset="validation",
batch_size=50,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 5000 validated image filenames belonging to 10 classes.


In [31]:
model_aug = Sequential()
model_aug.add(Conv2D(32, (3, 3),activation="relu" ,padding='same',input_shape=(32,32,3)))
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(Dropout(0.20))

model_aug.add(Conv2D(64, (3, 3), activation = "relu" ,padding='same'))
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(Dropout(0.20))

model_aug.add(Conv2D(128, (3, 3), activation = "relu" ,padding='same'))
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(Dropout(0.20))

model_aug.add(Flatten())
model_aug.add(Dense(512))
model_aug.add(Activation('relu'))
model_aug.add(Dropout(0.4))

model_aug.add(Dense(10, activation='softmax'))
model_aug.compile(optimizers="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model_aug.load_weights('model_DA.h5')

In [32]:
model_aug.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 128)        

In [33]:
STEP_SIZE_TRAIN_AUG=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID_AUG=valid_generator_aug.n//valid_generator.batch_size

In [34]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator_aug,
                    validation_steps=STEP_SIZE_VALID_AUG,
                    epochs=10
)

Epoch 1/10
500/500 [==============================] - 101s 203ms/step - loss: 1.1954 - accuracy: 0.5889 - val_loss: 0.8697 - val_accuracy: 0.7188
Epoch 2/10
500/500 [==============================] - 94s 189ms/step - loss: 1.1609 - accuracy: 0.6014 - val_loss: 0.7769 - val_accuracy: 0.7410
Epoch 3/10
500/500 [==============================] - 94s 188ms/step - loss: 1.1409 - accuracy: 0.6107 - val_loss: 0.8984 - val_accuracy: 0.7010
Epoch 4/10
500/500 [==============================] - 96s 192ms/step - loss: 1.1317 - accuracy: 0.6134 - val_loss: 0.8320 - val_accuracy: 0.7238
Epoch 5/10
500/500 [==============================] - 94s 189ms/step - loss: 1.1223 - accuracy: 0.6158 - val_loss: 0.8476 - val_accuracy: 0.7252
Epoch 6/10
500/500 [==============================] - 95s 189ms/step - loss: 1.1255 - accuracy: 0.6178 - val_loss: 0.8444 - val_accuracy: 0.7260
Epoch 7/10
500/500 [==============================] - 94s 187ms/step - loss: 1.1195 - accuracy: 0.6172 - val_loss: 0.8879 - val_a

In [35]:
model.load_weights('model_DA.h5')
x = model.save('model_DA.h5')
new_model = tf.keras.models.load_model('model_DA.h5')

In [36]:
from PIL import Image
from tensorflow import keras
import numpy as np

In [37]:
img = Image.open(r'C:\Users\M1048974\Desktop\pic\dog1.jpg')
img.show()

In [38]:
img = img.resize((32,32),Image.ANTIALIAS)
img_array = np.array(img)
img_array = img_array.astype('float32')
img_array /= 255
img_array = np.expand_dims(img_array,axis=0)
predict = new_model.predict(img_array)
predict

array([[0.1057284 , 0.10117178, 0.10969871, 0.09891579, 0.10011469,
        0.09259758, 0.09211371, 0.09483733, 0.12124507, 0.08357693]],
      dtype=float32)

In [41]:
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Ship', 'Frog', 'Horse', 'Dog', 'Truck']
index = (list(predict[0])).index(max((list(predict[0]))))
print("prediction is ---------------------->:",labels[index])

prediction is ---------------------->: Dog
